In [1]:
import os
os.chdir(os.path.join(os.path.abspath(os.path.expanduser("~")), "Documents", "Personal", "Test"))
from scripts.tools.issuetracker import IssuetrackerAPI as _ISAPI, Issue
import pickle

import datetime, dateutil
from_date = datetime.datetime(2016, 6, 1, tzinfo=dateutil.tz.tzutc())
today =datetime.date.today()
to_date = datetime.datetime(today.year, today.month, today.day, tzinfo=dateutil.tz.tzutc())


class IssuetrackerAPI(_ISAPI):
    
    def make_pickle_cache(self, *args):
        issues = super().download_issues(*args)
        with open("gantt_dummy.pkl", 'wb') as f:
            pickle.dump(issues, f)
        return issues
    def download_issues(self, *args):
        if not os.path.exists("gantt_dummy.pkl"):
            return self.make_pickle_cache(*args)
        else:
            with open("gantt_dummy.pkl", 'rb') as f:
                try:
                    issues = pickle.load(f)
                except (pickle.PickleError, OSError, EOFError):
                    pass
                else:
                    return issues
            return self.make_pickle_cache(*args)
        return issues


In [2]:
from officelib import wordlib
from scripts.tools.issuetracker.frs_update import main
import shutil
word = wordlib.Word()
pdfpth = "pdfs6"
try:
    word.ActiveDocument.Close(False)
except:
    pass
try:
    shutil.rmtree(pdfpth)
except FileNotFoundError:
    pass
print("Begin Main", flush=True)
res = main("issue.pbsbiotech.com", 'nstarkweather', 'kookychemist', 'pbssoftware', "2016-6-1", 
     "today", 'Hello v3.0 FRS Update', pdfpth, "FRS Update", 
     "IS00038B2 Rev A Functional Requirements Specification Updates")
#os.startfile('"%s"'%res)
print()
print("Done")

Begin Main

Logging into Issuetracker...Downloading projects...

Got update: PGS_BEGIN_PROCESS_PDFS 
Creating PDF Index...
Creating Word Document...
Creating index 1 of 34
Creating index 6 of 34                    
Creating index 9 of 34                    
Creating index 16 of 34                    
Creating index 22 of 34                    
Creating index 29 of 34                    
Creating index 34 of 34                    
Got update: PGS_IPG_FLAG_WAITING 
Got update: MPD_ALL_COMPLETE 
Got update: PGS_PDFTK_INFO_BEGIN 
Getting PDF Page Info (29/34)
Got update: PGS_IPG_FLAG_WAITING 2
Getting PDF Page Info (34/34)
Got update: PGS_PDFTK_INFO_DONE 
Got update: PGS_PROCESS_PDFS_DONE 
Got update: PGS_PREPARE_POSTSCRIPT 
Got update: PGS_BEGIN_BIND_PDF 
Got update: PGS_DONE 
Done


In [1]:
from scripts.tools.issuetracker import IssuetrackerAPI
import re

api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
issues = api.download_issues(project_id="pbssoftware")
filtered = [i for i in issues.values() if i.sprint_milestone.name == "3.0"]


magic = re.compile(r"[\+\*]+(3.0Web)?(FRS\d+[0-9\.]+)")
magic2 = re.compile(r"(3.0Web)?(FRS\d+[0-9\.]+)")

In [86]:
frs = set()
frs2 = set()

for v in filtered:
    d = v.description or ""
    l = magic.findall(d)
    for w, f in l:
        w = w or ""
        if "." not in f and v.assigned_to.name == "Christian Nilsson" and not w:
            w = "3.0Web"
        result = (v.id, w+f)
        frs.add(result)
    l = magic2.findall(d)
    for w, f in l:
        w = w or ""
        if "." not in f and v.assigned_to.name == "Christian Nilsson" and not w:
            w = "3.0Web"
        result = (v.id, w+f)
        frs2.add(result)
    
sorted(frs)

[(420, 'FRS420.0'),
 (977, 'FRS977.0'),
 (989, 'FRS989.0'),
 (1073, 'FRS1073.01'),
 (1073, 'FRS1073.02'),
 (1073, 'FRS1073.03'),
 (1073, 'FRS1073.11'),
 (1073, 'FRS1073.1304'),
 (1073, 'FRS1073.1305'),
 (1073, 'FRS1073.1306'),
 (1073, 'FRS1073.1307'),
 (1073, 'FRS1073.1308'),
 (1073, 'FRS1073.1309'),
 (1073, 'FRS1073.1310'),
 (1073, 'FRS1073.1311'),
 (1073, 'FRS1073.1312'),
 (1073, 'FRS1073.1313'),
 (1073, 'FRS1073.1314'),
 (1073, 'FRS1073.1315'),
 (1073, 'FRS1073.1316'),
 (1262, 'FRS1262.0'),
 (1263, 'FRS1263.0'),
 (1583, 'FRS1583.0'),
 (1583, 'FRS1583.1'),
 (1583, 'FRS1583.2'),
 (1583, 'FRS1583.3'),
 (1583, 'FRS1583.4'),
 (1754, 'FRS1754.0'),
 (1759, 'FRS1759.0'),
 (1845, 'FRS1845.0'),
 (1902, 'FRS1902.0'),
 (2115, 'FRS2115.0'),
 (2210, 'FRS2210.0'),
 (2303, 'FRS2303.0'),
 (2304, 'FRS2304.1'),
 (2304, 'FRS2304.10'),
 (2304, 'FRS2304.11'),
 (2304, 'FRS2304.12'),
 (2304, 'FRS2304.13'),
 (2304, 'FRS2304.14'),
 (2304, 'FRS2304.15'),
 (2304, 'FRS2304.16'),
 (2304, 'FRS2304.17'),
 (2304, '

In [158]:
from scripts.tools.issuetracker import IssuetrackerAPI
import re
try:
    cache
except NameError:
    cache = None
cache = cache or {}
def extract_frs_items():
    global cache
    magic = re.compile(r"[\+\*\s]*(3.0Web)?(FRS\d+[0-9\.]+)[\+\*: \-]*")
    if not cache:
        api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
        iss = api.download_issues(project_id="pbssoftware")
        cache = iss
    else:
        iss = cache
    filtered = [i for i in iss.values() if i.sprint_milestone.name == "3.0"]
    
    reqs = []
    reqs2 = []
    frs = set()
    frs2 = set()
    for v in filtered:
        d = v.description or ""
        matches = magic.findall(d)
        for web, fr in matches:
            web = web or ""
            result = v.id, web + fr
            reqs.append(result)
            frs.add(web + fr)
        lines = d.split("\r\n")
        for l in lines:
            m = magic.search(l)
            if m:
                web, fr = m.groups()
                web = web or ""
                rest = l[m.end():]
                result = (v.id, web + fr, rest)
                reqs2.append(result)
                frs2.add(web+fr)

    reqs.sort()
    reqs2.sort()
    return reqs, reqs2, frs, frs2

reqs, reqs2, frs, frs2 = extract_frs_items()
with open("test.csv", 'w') as f:
    for line in reqs2:
        f.write(", ".join(map(str, line)))
        f.write(",\n")

In [174]:
from scripts.tools.issuetracker import IssuetrackerAPI
import re
try:
    cache
except NameError:
    cache = None
cache = cache or {}
magic = re.compile(r"[\+\*\s]*(3.0Web)?(FRS\d+[0-9\.]+)[\+\*: \-]*")

def get_issues():
    global cache
    if not cache:
        api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
        iss = api.download_issues(project_id="pbssoftware")
        cache = iss.copy()
    else:
        iss = cache.copy()
    return iss

def extract_frs_items2():
    iss = get_issues()
    filtered = [i for i in iss.values() if i.sprint_milestone.name == "3.0"]
    reqs = []
    frs = set()
    for v in filtered:
        d = v.description or ""
        lines = d.split("\r\n")
        i = 0
        while i < len(lines):
            l = lines[i]
            m = magic.search(l)
            if m:
                web, fr = m.groups()
                web = web or ""
                rest = l[m.end():]
                other = []
                i += 1
                while i < len(lines):
                    l = lines[i]
                    if not l.strip():
                        break
                    elif magic.search(l):
                        break
                    else:
                        other.append(l.strip())
                    i += 1                        
                result = (v.id, web + fr, rest, "\n".join(other))
                reqs.append(result)
            i += 1    
    reqs.sort()
    return reqs

reqs= extract_frs_items2()
with open("test.txt", 'w') as f:
    for id, fr, rest, other in reqs:
        f.write("*%s*%s\n"%(fr, ": %s" % rest if rest else ""))
        f.write(other)
        f.write("\n\n")
import os; os.startfile(f.name)




X" have actions which are in the Shell




X" use the Webservice Event to DB sub vi and are in the Webservice Command Handler















Changes to Enable Disable and Calibrate VI
Changes to Pick a Sensor VI
Changes to Control Module VIs (Temp, pH and DO)
Changes to Set Sensor States VI
Changes to Webservice Command Handler





Functionality

Any Account
Settings Editor
System Management
Acknowledge Alarms
Any Client
Batch
Door
Hello Access
Hello Settings
Light
Pumps
Account Management
Recipe End
Recipe Start
Remote Hello Access
Sensors
Advanced Calibration
Alarms Editor
Controls
DB Management
Logger Editor
Recipe Editor
Reports



















When the Web UI is open and the network cable is disconnected, the web server should return blank data to the Web UI, so it doesn't force a reload.  All displayed data values will be 0.
Correct the getVersions call and the RIO Serial Number indicator in the LabVIEW UI's "About" layout to show the RIO serial number correctly
Make all a